In [1]:
from helpers import *
from models import *
from sklearn import *

//anaconda/envs/cancer/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
//anaconda/envs/cancer/lib/python3.6/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)
//anaconda/envs/cancer/lib/python3.6/site-packages/sklearn/lda.py:6: DeprecationWarning: lda.LDA has been moved to discriminant_analysis.LinearDiscriminantAnalysis in 0.17 and will be removed in 0.19
  "in 0.17 and will be removed in 0.19", DeprecationWarnin

In [2]:
train_full = get_full_table('/Users/yao/Desktop/ML/cancer/I_vote_for_cancer/input/training_variants', \
             '/Users/yao/Desktop/ML/cancer/I_vote_for_cancer/input/training_text')

#dummy_full, test_full = model_selection.train_test_split(\
#     data_full, test_size=0.2, random_state=42, stratify=data_full['Class']-1)

#train_full, val_full = model_selection.train_test_split(\
#     dummy_full, test_size=0.2, random_state=42, stratify=dummy_full['Class']-1)

#print(train_full['Class'].describe())
#print(val_full['Class'].describe())
#print(test_full['Class'].describe())
pl1 = GeneSvm().fit(train_full)

this is a model based on only gene&variations


In [ ]:
#GeneSvm().predict_proba(pl1, train_full)
#feature1 = GeneSvm().predict_proba(pl1, val_full)
#GeneSvm().predict_proba(pl1, test_full)

In [5]:
pl2 = TfidfSvm().fit(train_full)

this is a tfidf+svm model


In [3]:
#TfidfSvm().predict_proba(pl2, train_full)
#feature2 = TfidfSvm().predict_proba(pl2, val_full)
#TfidfSvm().predict_proba(pl2, test_full)

In [6]:
test_full = get_full_table('/Users/yao/Desktop/ML/cancer/I_vote_for_cancer/input/test_variants', \
             '/Users/yao/Desktop/ML/cancer/I_vote_for_cancer/input/test_text')

In [7]:
import numpy as np
X_train = np.concatenate((GeneSvm().predict_proba(pl1, train_full),\
                          TfidfSvm().predict_proba(pl2, train_full)), axis=1)
y_train = train_full['Class']-1

X_test = np.concatenate((GeneSvm().predict_proba(pl1, test_full), TfidfSvm().predict_proba(pl2, test_full)), axis=1)
#y_test = test_full['Class']-1

this is a model based on only gene&variations
logloss:
0.695261760578
this is a tfidf+svm model
logloss:
0.84096230763
this is a model based on only gene&variations
this is a tfidf+svm model


In [8]:
##Fitting
parameters = {
    "estimator__C": [10],
    "estimator__kernel": ['rbf'],
    #"estimator__degree": [2, 3]
}

clf = GridSearchCV(OneVsRestClassifier(svm.SVC(probability=True, class_weight='balanced')),\
                   param_grid=parameters, scoring='neg_log_loss', n_jobs=-1)
clf.fit(X_train, y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=OneVsRestClassifier(estimator=SVC(C=1.0, cache_size=200, class_weight='balanced', coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=True, random_state=None, shrinking=True,
  tol=0.001, verbose=False),
          n_jobs=1),
       fit_params={}, iid=True, n_jobs=-1,
       param_grid={'estimator__C': [10], 'estimator__kernel': ['rbf']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='neg_log_loss', verbose=0)

In [ ]:
pd.DataFrame.from_dict(clf.cv_results_)

In [9]:
y_train_prob = clf.predict_proba(X_train)
y_test_prob = clf.predict_proba(X_test)
print(log_loss(y_train, y_train_prob, eps=1e-15, normalize=True, labels=range(9)))
submission('./input/submission_ensemble1.csv', y_test_prob)
#print(log_loss(y_test, y_test_prob, eps=1e-15, normalize=True, labels=range(9)))

0.432969825797


In [ ]:
df = pd.DataFrame(y_test_prob, columns=['class'+str(c+1) for c in range(9)])
df['Class_pred'] = df.idxmax(axis=1).map(lambda x:int(x[5]))
df['ID'] = test_full['ID'].values

In [ ]:
df = df.merge(data_full, how='inner', on='ID')
df_wrong = df[df['Class_pred']!= df['Class']]

In [ ]:
df_wrong['Gene'].unique()